# Bank Deposit Prediction

Importing libraries

In [75]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

Importing Data:
1. In this case the Training and Testing Data were already in seperated files
<br>
2. There was no missing data
<br>
3. Dummy columns were created


In [76]:
BankDepositsTrain = pd.read_csv('C:/Users/Jody/Documents/data sets/Bank Term Deposit Prediction/train.csv')
BankDepositsTrain1 = pd.get_dummies(BankDepositsTrain, columns = ['job','marital','education', 'default', 'poutcome','loan','y'],
                                   dtype = int)


BankDepositsTest = pd.read_csv('C:/Users/Jody/Documents/data sets/Bank Term Deposit Prediction/test.csv')
BankDepositsTest1 = pd.get_dummies(BankDepositsTest, columns = ['job','marital','education', 'default', 'poutcome','loan','y'],
                                   dtype = int)



train = BankDepositsTrain1.drop(['housing','contact','day','month','duration','campaign','pdays','previous','y_no'],axis = 1)

test = BankDepositsTest1.drop(['housing','contact','day','month','duration','campaign','pdays','previous','y_no'],axis = 1)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   age                  4521 non-null   int64
 1   balance              4521 non-null   int64
 2   job_admin.           4521 non-null   int32
 3   job_blue-collar      4521 non-null   int32
 4   job_entrepreneur     4521 non-null   int32
 5   job_housemaid        4521 non-null   int32
 6   job_management       4521 non-null   int32
 7   job_retired          4521 non-null   int32
 8   job_self-employed    4521 non-null   int32
 9   job_services         4521 non-null   int32
 10  job_student          4521 non-null   int32
 11  job_technician       4521 non-null   int32
 12  job_unemployed       4521 non-null   int32
 13  job_unknown          4521 non-null   int32
 14  marital_divorced     4521 non-null   int32
 15  marital_married      4521 non-null   int32
 16  marital_single       452

There was a very large class imbalance where class 1 only made up 12% of the dataset.
<br>
Over sampling was utilized to mitigate this

In [77]:
class_0 = train[train['y_yes'] == 0]
class_1 = train[train['y_yes'] == 1]

print(class_0.shape, class_1.shape)

(39922, 30) (5289, 30)


In [78]:
Class0regularsample = class_0
Class1regularsample = class_1.sample(39922, replace = True)

trainresampled = pd.concat([Class0regularsample,Class1regularsample], axis = 0)

X and Y were defined. Again the train and test dataset were already provided so there was no use
<br>
for the train test split function

In [79]:
X_train = trainresampled.drop(['y_yes'],axis = 1)
Y_train = trainresampled['y_yes']
X_test = test.drop(['y_yes'],axis = 1)
Y_test = test['y_yes']

GridsearchCV was utlized for optimal model tuning

In [80]:
dt_classifier = DecisionTreeClassifier()

param_grid = {
    'criterion': ['entropy', 'gini'],
    'max_depth': [None, 5, 10, 15, 20, 25], 
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4,6],
}

grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, Y_train)


best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print("Best Parameters:", best_params)

Best Parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [ ]:
y_pred = best_estimator.predict(X_test)

Classification Report

In [83]:
cr = classification_report(Y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4000
           1       0.93      1.00      0.96       521

    accuracy                           0.99      4521
   macro avg       0.96      0.99      0.98      4521
weighted avg       0.99      0.99      0.99      4521

